# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *H*

**Names:**

* *BAFFOU Jérémy*
* *BASSETO Antoine*
* *PINTO Andrea*

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.2 Network sampling

### Exercise 2.7: Random walk on the Facebook network

Crawl the Facebook graph to estimate the average age of users in the social network using the random walk approach of Algorithm 2.1, starting from node "a5771bce93e200c36f7cd9dfd0e5deaa". What is your estimation of the average age of a Facebook user? How many users did you visit to get this estimation? Hint: Refer to the helper notebook ix−lab2−helper.ipynb to get an example code illustrating how to query the API.

In [1]:
import random
import requests
import numpy as np

In [2]:
URL_TEMPLATE = 'http://iccluster041.iccluster.epfl.ch:5050/v1.0/facebook?user={user_id}'
start_uid = 'a5771bce93e200c36f7cd9dfd0e5deaa'

In [5]:
def request(id):
    url = URL_TEMPLATE.format(user_id=id)
    response = requests.get(url)
    return response.json()

In [7]:
data = request(start_uid)

In [8]:
data['age']

13

In [11]:
data['friends'][0:5]

['0065470d21f5f095cc304a048984387a',
 '634f079997d1329f9504e68870b33a9f',
 '6005f35032ccbc32c007209c9547a709',
 '08f3a0aaa93613f038951a575d280c7f',
 'c55d0ff1eae9cddaf7210e9c71723525']

In [12]:
def infos(users):
    print(f'Crawled {len(users)} nodes in the network')
    print(f'From which {len(set(users))} are unique users')

In [49]:
def average_age_naive(users, ages):
    uniques, indices = np.unique(users, return_index=True)
    filtered_ages = np.array(ages)[indices]
    return np.sum(filtered_ages) / len(uniques)

In [51]:
def average_age_smart(users, ages, degrees):
    uniques, indices = np.unique(users, return_index=True)
    filtered_ages = np.array(ages)[indices]
    filtered_degrees = np.array(degrees)[indices]
    return np.sum(filtered_ages / filtered_degrees) / np.sum(1/filtered_degrees)

In [56]:
def average_age_smart_2(users, ages, degrees):
    return np.sum(ages / degrees) / np.sum(1/degrees)

In [31]:
# fake shit
def get_next_uid(uid, friends):
    succ = random.choice(friends)
    while succ != uid:
        succ = random.choice(friends)
    return succ

In [45]:
def random_walk(sid, iters):
    data = request(sid)
    
    users = []
    ages = []
    degrees = []
    
    users.append(sid)
    ages.append(data['age'])
    degrees.append(len(data['friends']))
    
    epoch = 1
    uid = random.choice(data['friends'])
    
    while epoch < iters: # uid != sid 
        data = request(uid)
        users.append(uid)
        ages.append(data['age'])
        degrees.append(len(data['friends']))
        uid = random.choice(data['friends'])
        epoch += 1
        print(epoch, end='\r')
    
    return users, ages, degrees

In [60]:
users, ages, degrees = naive_random_walk(start_uid, 1000)

In [61]:
infos(users)
print(average_age_naive(users, ages))
print(average_age_smart(users, ages, degrees))
print(average_age_smart_2(np.array(users), np.array(ages), np.array(degrees)))

Crawled 1000 nodes in the network
From which 754 are unique users
20.19628647214854
39.199671292348036
43.50092458127409
